# Estimating quartiles

# In brevi

The purpose of this notebook is to explore the ways that Python calculates quartiles. During the development of the anova_one_factor notebook, I discovered that Python, Libre Office, and Excel calculate quartiles in the same way, but that Minitab and an online source calculate them differently. I've discovered that there are at least nine ways to calculate quartiles.

# Data

Download the data file.

[estimating_quartiles](https://drive.google.com/open?id=1Nc_VFXo2SrsSdprfCmQYhLbJawAzKpH6)

# Methodology

Various data munging operations are performed using pandas.

In [1]:
from datetime import datetime
start_time = datetime.now()

In [2]:
# Import the required librairies.
import pandas as pd

/home/gilles/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gilles/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# Read the data file.
# y is the column of response values.
df = pd.read_csv('estimating_quartiles.csv')

In [4]:
#def five_number_summaries(data, inter):
#    global result
#    for v in inter:
#        print(v)
##    min = data.min()
##    q1 = data.quantile(.25, interpolation=v)
##    q2 = data.quantile(.50, interpolation=v)
##    q3 = data.quantile(.75, interpolation=v)
##    max = data.max()
##    five_numbers = {
##    'min' : [min],
##    'q1' : [q1],
##    'q2' : [q2],
##    'q3' : [q3],
##    'max' : [max]
##    }
##    result = pd.DataFrame(five_numbers)
##    result.append(other)
#    return result

In [5]:
a = []          ## Start as the empty list
a.append('a')   ## Use append() to add elements
a.append('b')
a

['a', 'b']

In [6]:
#def five_number_summaries(data, inter):
#    df = []
#    for i in inter:
#        min = data.min()
#        q1 = data.quantile(.25, interpolation=i)
#        q2 = data.quantile(.50, interpolation=i)
#        q3 = data.quantile(.75, interpolation=i)
#        max = data.max()
#        df.append(min)
#        df.append(q1)
#        df.append(q2)
#        df.append(q3)
#        df.append(max)
#    df = pd.DataFrame(df)
#    return df

In [7]:
#def five_number_summaries(data, inter):
#    df = pd.DataFrame([])
#    for i in inter:
#        min = data.min()
#        q1 = data.quantile(.25, interpolation=i)
#        q2 = data.quantile(.50, interpolation=i)
#        q3 = data.quantile(.75, interpolation=i)
#        max = data.max()
#        five_numbers = {
#        'min' : [min],
#        'q1' : [q1],
#        'q2' : [q2],
#        'q3' : [q3],
#        'max' : [max]
#        }
#        df = df.append(five_numbers, ignore_index=True)
#        df = df.append(pd.DataFrame(five_numbers), ignore_index=True)
#    return df

In [8]:
def five_number_summaries(data: pd.Series) -> pd.DataFrame:
   return pd.DataFrame([(interpolation,
                         data.min(),
                         data.quantile(0.25, interpolation=interpolation),
                         data.quantile(0.50, interpolation=interpolation),
                         data.quantile(0.75, interpolation=interpolation),
                         data.max())
                        for interpolation
                        in ('linear', 'lower', 'higher', 'nearest', 'midpoint')],
                        columns=['interpolation', 'min', 'q1', 'q2', 'q3', 'max']).\
                        set_index(['interpolation'])


In [9]:
results = five_number_summaries(df['y'])
results

,min,q1,q2,q3,max
interpolation,,,,,
linear,0,0.75,7.5,35.5,63
lower,0,0.00,2.0,27.0,63
higher,0,1.00,13.0,61.0,63
nearest,0,1.00,13.0,27.0,63
midpoint,0,0.50,7.5,44.0,63


In [10]:
results.iloc[0,3]

35.5

In [11]:
df.describe()

,y
count,8.000000
mean,20.875000
std,27.010249
min,0.000000
25%,0.750000
50%,7.500000
75%,35.500000
max,63.000000


In [12]:
end_time = datetime.now()
(end_time - start_time).total_seconds()

0.470579

# References

[Five-number summary](https://en.wikipedia.org/wiki/Five-number_summary)